# Softmax Classification_Theoretical Overview

Logistic Regression이 input에 대한 연산 결과를 0~1 사이의 확률값, 그리고 두 가지 중에 하나의 결론을 내리는 방법이었다면,

Softmax Classification은 다중 분류를 해결하기 위한 모델이다.

input : 1개 이상, ouput : 2개 이상

이 때, 여러 개의 연산 결과를 정규화하여 모든 클래스의 확률값 합을 1로 만들고자 한다.


$$ P(class=i) = \frac{e^i}{\sum e^i} $$

# Softmax Classification_basic

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# For reproducibility
torch.manual_seed(1)

In [2]:
# 바로 의사결정
z = torch.FloatTensor([1, 2, 3])
print(max(z))

# 확률값으로 의사결정
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

# 확률값의 합은 1
hypothesis.sum()

tensor(3.)
tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

## 실습

두 개의 확률분포가 주어져있을 때 두 확률분포가 얼마나 비슷한지 알아보자

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

$$ y = P(x) $$
$$\hat{y} = Q(x) $$ 

In [20]:
z = torch.rand(3, 5, requires_grad=True) # ( 3, 5 )

hypothesis = F.softmax(z, dim=1)
print(hypothesis)

y = torch.randint(5, (3,)).long()
print(y)

tensor([[0.2528, 0.1500, 0.2019, 0.1292, 0.2662],
        [0.1518, 0.2729, 0.1887, 0.1441, 0.2425],
        [0.1761, 0.2659, 0.1987, 0.2173, 0.1420]], grad_fn=<SoftmaxBackward>)
tensor([0, 3, 0])


In [21]:
# 위의 결과값을 (index로 표현되어있음) one-hot vector로 표현하자

# 같은 size의 0으로 구성된 것 vector 생성
y_one_hot = torch.zeros_like(hypothesis)

# dimension=1에 대해서 y를 unsqueeze해서 vector크기를 맞춰주자
# unsqueeze : (3, ) => (3, 1)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])

In [22]:
# y_one_hot : (3, 5)
# torch.log(hypothesis) : (3, 5)
# sum(dim=1) : (3, 1)
# mean() : (1, 1)

cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6830, grad_fn=<MeanBackward0>)


## 실습2

In [17]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [18]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True) # 4차원 벡터 n개 , 예측할 값 3개
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 1.098612
Epoch 1000/10000 Cost: 0.738749
Epoch 2000/10000 Cost: 0.710759
Epoch 3000/10000 Cost: 0.697990
Epoch 4000/10000 Cost: 0.690640
Epoch 5000/10000 Cost: 0.685888
Epoch 6000/10000 Cost: 0.682576
Epoch 7000/10000 Cost: 0.680144
Epoch 8000/10000 Cost: 0.678285
Epoch 9000/10000 Cost: 0.676820
Epoch 10000/10000 Cost: 0.675637


### optim.SGD

# Softmax Classification_nn.Module

## Data

In [24]:
z = torch.rand(3, 5, requires_grad=True) # ( 3, 5 )

hypothesis = F.softmax(z, dim=1)
print(hypothesis)

y = torch.randint(5, (3,)).long()
print(y)

tensor([[0.2659, 0.2413, 0.1579, 0.1158, 0.2191],
        [0.1887, 0.1170, 0.2236, 0.2189, 0.2518],
        [0.1707, 0.2080, 0.2237, 0.2430, 0.1546]], grad_fn=<SoftmaxBackward>)
tensor([3, 4, 3])


## one-hot vector

In [25]:
# 위의 결과값을 (index로 표현되어있음) one-hot vector로 표현하자

# 같은 size의 0으로 구성된 것 vector 생성
y_one_hot = torch.zeros_like(hypothesis)

# dimension=1에 대해서 y를 unsqueeze해서 vector크기를 맞춰주자
# unsqueeze : (3, ) => (3, 1)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.]])

## cost

In [26]:
# y_one_hot : (3, 5)
# torch.log(hypothesis) : (3, 5)
# sum(dim=1) : (3, 1)
# mean() : (1, 1)

# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

# 이것도 한 번에 할 수 있다
F.cross_entropy(z, y)

tensor(1.6500, grad_fn=<MeanBackward0>)


## train

In [27]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

In [29]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.248546
Epoch 1000/10000 Cost: 0.158548
Epoch 2000/10000 Cost: 0.116344
Epoch 3000/10000 Cost: 0.091576
Epoch 4000/10000 Cost: 0.075368
Epoch 5000/10000 Cost: 0.063968
Epoch 6000/10000 Cost: 0.055526
Epoch 7000/10000 Cost: 0.049030
Epoch 8000/10000 Cost: 0.043881
Epoch 9000/10000 Cost: 0.039701
Epoch 10000/10000 Cost: 0.036242
